# XSS Detection in form data using Machine Learning for attack discovery and mitigation 

## Information
- Kaggle XSS dataset (https://github.com/agentsmith1337/AgenticWAF/blob/main/Models/XSS/XSS_dataset.csv)
- Focus on SVMs and Forest Models
- XSS Detection can be done by detecting javascript and html alone, but this is a good broader starting point.
- Names and generic text has to be augmented into this data since real traffic rarely has code snippets as user input. 

In [76]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://raw.githubusercontent.com/agentsmith1337/AgenticWAF/main/Models/XSS/XSS_dataset.csv"

dat=pd.read_csv(url, encoding="unicode_escape")
dat.head()

,ï»¿,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [77]:
dat.shape

(13686, 3)

In [78]:
dat.columns

Index(['ï»¿', 'Sentence', 'Label'], dtype='object')

In [79]:
dat.info

<bound method DataFrame.info of          ï»¿                                           Sentence  Label
0          0  <li><a href="/wiki/File:Socrates.png" class="i...      0
1          1               <tt onmouseover="alert(1)">test</tt>      1
2          2  \t </span> <span class="reference-text">Steeri...      0
3          3  \t </span> <span class="reference-text"><cite ...      0
4          4  \t </span>. <a href="/wiki/Digital_object_iden...      0
...      ...                                                ...    ...
13681  13681             <img onpointerenter=alert(1)>XSS</img>      1
13682  13682  <source onbeforepaste="alert(1)" contenteditab...      1
13683  13683  <div draggable="true" contenteditable>drag me<...      1
13684  13684  <li><cite id="CITEREFDomingos2015" class="cita...      0
13685  13685                                         \t </span>      0

[13686 rows x 3 columns]>

We have to add nearly 8000 data rows consisting of generic text

In [80]:
from faker import Faker
import random
fake=Faker()
aug=[]
for i in range(2000):
    choice=random.choice([fake.name, fake.address, fake.sentence, fake.email])
    aug.append({
        'text': choice(),
        'label': 0
    })
dat0=pd.DataFrame(aug)
dat0.head()
    

,text,label
0,Christopher Edwards,0
1,"0762 Michael Fork\nBradleytown, AZ 50325",0
2,ahernandez@example.net,0
3,tracy60@example.com,0
4,Current peace air.,0


In [81]:
dat=dat.iloc[:, 1:]
dat.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [82]:
dat1=dat.rename(columns={'Sentence':'text','Label':'label'})
dat1['text']=dat1['text'].str.strip()
dat1.head()

,text,label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"</span> <span class=""reference-text"">Steering ...",0
3,"</span> <span class=""reference-text""><cite cla...",0
4,"</span>. <a href=""/wiki/Digital_object_identif...",0


In [83]:
data=pd.concat([dat1, dat0], ignore_index=True)
data.head()

,text,label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"</span> <span class=""reference-text"">Steering ...",0
3,"</span> <span class=""reference-text""><cite cla...",0
4,"</span>. <a href=""/wiki/Digital_object_identif...",0


In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15686 entries, 0 to 15685
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15686 non-null  object
 1   label   15686 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 245.2+ KB


In [85]:
df=data.copy()

In [86]:
# 1. Define the hard negatives (benign text that looks scary)
hard_negatives = [
    "To fix the error, remove the <script> tag from your header.",
    "The alert() function in JavaScript is used for debugging.",
    "Do not use onerror handlers blindly in production code.",
    "I was reading about XSS attacks and how <img src=x> works.",
    "The onload event fires when the window has finished loading.",
    "You can use document.cookie to access cookies safely.",
    "Review the security policy regarding script execution.",
    "The string '<script>' is just text if properly escaped.",
    "Check out this link: http://example.com/search?q=script",
    "Writing a tutorial on how to prevent alert(1) popups.",
    "Is it safe to use innerHTML if I sanitize the input first?",
    "those who 'script alert onload are <geeks>",
    "script alert onload onerror",
    "prompt her the document.cookie",
    "my cookie is stuck in the document",
    "ask her onerror script load <don't ask me>" # Your specific example
    # Add more technical documentation style sentences here if needed...
]

# 2. Create a DataFrame for these new samples
df_hard_negatives = pd.DataFrame({
    'text': hard_negatives,
    'label': 0  # Label these as BENIGN (0)
})

# 3. Merge with your existing 'df'
df = pd.concat([df, df_hard_negatives], ignore_index=True)

print(f"Added {len(df_hard_negatives)} hard negative samples.")
print("New dataset size:", df.shape)

Added 16 hard negative samples.
New dataset size: (15702, 2)


# Feature Engineering with TF-IDF for weightage of words in relation to the interesting class

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#--------------------------------------------------
# 1. Data Split
#--------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

#--------------------------------------------------
# 2. Create a Pipeline (Vectorizer + Model)
# Key setting: analyzer='char' is CRITICAL for XSS to catch partial tags like '<scr'
#--------------------------------------------------

pipeline = make_pipeline(
    TfidfVectorizer(analyzer='char', ngram_range=(2, 5), max_features=10000),
    SVC(kernel='linear', probability=True) # Linear Kernel for projection
)
#--------------------------------------------------
# 3. Train
pipeline.fit(X_train, y_train)
#--------------------------------------------------

#--------------------------------------------------
# 4. Evaluate
#--------------------------------------------------
print(classification_report(y_test, pipeline.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1659
           1       1.00      1.00      1.00      1482

    accuracy                           1.00      3141
   macro avg       1.00      1.00      1.00      3141
weighted avg       1.00      1.00      1.00      3141



In [88]:
df['label'].value_counts()
# df.head(10)

label
0    8329
1    7373
Name: count, dtype: int64

In [89]:
# Define your own inputs to test
my_inputs = [
    "<script>alert('XSS')</script>",      # Obvious XSS
    "Hello, how are you?",                # Benign text
    "<img src=x onerror=alert(1)>",       # Event handler XSS
    "javascript:alert(1)",    
    "<image src =q onerror=prompt(8)>",
    "I should say, you look good today!",
    "What the ^%&^$% how could you!!???",
    "<script>/* *\x00/javascript:alert(1)// */</script>",
    "(though i am used to it) she's the one who ^&' script it",
    "those who 'script alert onload are <geeks>",            # Protocol handler
    "script alert onload onerror",
    "<b>Bold Text</b>",
    "prompt script onload",
    "prompt script onload document.cookie",
    "google onerror <script> src tags"                    # HTML but likely benign
]
my_inputs=my_inputs*5

# Get predictions (0 = Benign, 1 = Malicious)
predictions = pipeline.predict(my_inputs)
probs = pipeline.predict_proba(my_inputs) # Get confidence scores

# Print results
for text, label, prob in zip(my_inputs, predictions, probs):
    status = "MALICIOUS" if label == 1 else "BENIGN"
    confidence = prob[label] * 100
    print(f"[{status}] ({confidence:.2f}%) - {text}")

[MALICIOUS] (100.00%) - <script>alert('XSS')</script>
[BENIGN] (100.00%) - Hello, how are you?
[MALICIOUS] (100.00%) - <img src=x onerror=alert(1)>
[MALICIOUS] (100.00%) - javascript:alert(1)
[MALICIOUS] (95.93%) - <image src =q onerror=prompt(8)>
[BENIGN] (99.98%) - I should say, you look good today!
[BENIGN] (100.00%) - What the ^%&^$% how could you!!???
[MALICIOUS] (100.00%) - <script>/* * /javascript:alert(1)// */</script>
[BENIGN] (99.99%) - (though i am used to it) she's the one who ^&' script it
[BENIGN] (99.59%) - those who 'script alert onload are <geeks>
[BENIGN] (36.69%) - script alert onload onerror
[BENIGN] (99.57%) - <b>Bold Text</b>
[BENIGN] (99.30%) - prompt script onload
[BENIGN] (46.37%) - prompt script onload document.cookie
[MALICIOUS] (100.00%) - google onerror <script> src tags
[MALICIOUS] (100.00%) - <script>alert('XSS')</script>
[BENIGN] (100.00%) - Hello, how are you?
[MALICIOUS] (100.00%) - <img src=x onerror=alert(1)>
[MALICIOUS] (100.00%) - javascript:alert(

In [91]:
import joblib
joblib.dump(pipeline, "C:/routines/agenticwaf/models/xss/xss_detection_model.pkl")


['C:/routines/agenticwaf/models/xss/xss_detection_model.pkl']